In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from pathlib import Path

# check if workding_dir is in local variables
if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)

os.chdir(workding_dir)
sys.path.append(workding_dir)
print("workding dir:", workding_dir)

In [ ]:
from dotenv import find_dotenv, load_dotenv

found_dotenv = find_dotenv(".env")

if len(found_dotenv) == 0:
    found_dotenv = find_dotenv(".env.example")
print(f"loading env vars from: {found_dotenv}")
load_dotenv(found_dotenv, override=True)

In [ ]:
from llm_toolkit.data_utils import *

data_path = os.getenv("DATA_PATH")
results_path = "results/amazon_reviews_results_all.csv"  # os.getenv("RESULTS_PATH")
num_ctx = os.getenv("NUM_CTX")
data_path, results_path, num_ctx

In [ ]:
import pandas as pd

df = pd.read_csv(results_path)
df.info()

In [ ]:
model_orders = {"deepseek-r1:32b": 1, "deepseek-r1:70b": 2}

In [ ]:
result_col_start_idx = 3
selected_result_col_start_idx = 3

In [ ]:
def get_key(model_name):
    parts = model_name.split("/")
    key = f"{int(model_orders[parts[0]] * 10):03d}/{parts[1]}"
    # print(key)
    return key


result_cols = [
    col
    for col in df.columns[selected_result_col_start_idx:].tolist()
    if col.split("/")[0] in model_orders.keys()
]
result_cols.sort(key=get_key)
result_cols

In [ ]:
df = df[df.columns[:result_col_start_idx].tolist() + result_cols]

In [ ]:
datasets = prepare_dataset(data_path, "Text", "Review-sentiment")

In [ ]:
metrics = get_metrics(
    df,
    result_col_start_idx=result_col_start_idx,
    mean_eval_time=True,
    train_dataset=datasets["train"],
    debug=True,
)
metrics

In [ ]:
metrics[metrics["model"].str.contains("32b")]["mean_num_tokens"].mean(), metrics[
    metrics["model"].str.contains("70b")
]["mean_num_tokens"].mean()